<a href="https://colab.research.google.com/github/RegNLP/ContextAware-Regulatory-GraphRAG-ObliQAMP/blob/main/05_Experiment_2_Graph_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**🔁 Enriched Graph Re-ranking with Champion Retrievers**

This section enhances the top-performing retriever configurations (from Experiment 1) by applying graph-based re-ranking strategies. It incorporates structural relationships—such as parent-child, citation links, and PageRank centrality—into the scoring of retrieved passages to improve final retrieval quality.

**✅ Evaluation Steps**

* 1️⃣ Setup and Configuration

  * Install required libraries: sentence-transformers, rank_bm25, pytrec_eval, etc.

  * Define paths for:

    * Regulatory graph (graph.gpickle)

    * Test question set (ObliQA_MultiPassage_test.json)

    * QREL file (qrels.trec)

    * Precomputed embeddings and output directories

* 2️⃣ Load Data

    * Load the graph structure containing documents and passages.

    * Load test questions and prepare a lookup (qid → question).

    * Load QRELs for evaluation.

    * Initialize a BM25 retriever for lexical scoring.

* 3️⃣ Precompute Graph Metrics

    * Run PageRank on the full graph to use as a centrality signal.

    * Store PageRank scores for all nodes.

* 4️⃣ Define Evaluation Helpers

    * add_instruction_to_query: Adds prompts like "query: " for E5-style models.

    * evaluate_run: Computes Recall@10, MAP@10, and nDCG@10 using pytrec_eval.

    * reciprocal_rank_fusion: Fuses dense and BM25 results into a hybrid ranking.

    * format_run_for_json: Converts ranked results to a readable JSON format.

* 5️⃣ Initial Retrieval with Champion Retrievers

    * For each selected retriever and context configuration:

      * Load the retriever and its passage embeddings.

      * For each query:

        * Encode the query and compute top-𝑘 dense scores.

        * Combine with BM25 top-𝑘 results using RRF to get initial hybrid results.

      * Store:

        * Top-100 passage candidates and initial scores

* 6️⃣ Graph-Based Re-Ranking

    * For each of the 144 hyperparameter combinations:

      * Apply the following graph-based signals to re-score top-100 passages:

        * Parent Bonus: if the passage’s parent is also retrieved

        * Sibling Bonus: if other passages share the same parent

        * CITES / CITED_BY Bonus: if linked nodes are also retrieved

        * PageRank Weight: centrality-based bonus

        * Isolation Penalty: penalty for nodes with no structural connections

      * Compute:
        
        Final Score= Initial Score + Graph Bonus + PageRank Bonus

      * Re-rank based on these scores.

      * Select top-20 passages for each question.

* 7️⃣ Evaluation and Output

    * For each configuration:

      * Evaluate the graph-re-ranked results using:

        * Recall@10

        * MAP@10

      * Save:

        * Metrics to a central DataFrame

        * Retrieved passage texts and scores as a JSON file
(filename encodes the hyperparameter settings)

* 8️⃣ Aggregate and Report Final Results

  * a. Combine All Metrics

    Store all configurations and their metrics in a single pandas DataFrame.

  * b. Sort and Rank

    Rank configurations by descending Recall@10 and MAP@10.

  * c. Save Final Outputs

    Save the final summary as a CSV:
      * experiment_2_final_graph_rerank_results.csv

      * Save 288+ JSON files: one per configuration and retriever

      * Print top-ranked configurations in the notebook

In [2]:
# ==============================================================================
# Experiment 2 (Final Run): Enriched Graph Re-ranking with Champion Retrievers
#
# Purpose:
# 1. Use the definitive champion fine-tuned retrievers from the final
#    retriever evaluation.
# 2. Systematically test multiple graph re-ranking strategies to see if they can
#    improve upon this new, stronger baseline.
# 3. Save both a CSV summary and detailed JSON outputs for each run.
# ==============================================================================

# --- Essential Installations ---
!pip install -q -U sentence-transformers transformers datasets rank_bm25 pytrec_eval

import os
import json
import torch
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import pytrec_eval
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
import itertools

# --- Configuration ---
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/RIRAG-MultiPassage-NLLP/"
GRAPH_PATH = os.path.join(BASE_PATH, "graph.gpickle")
TEST_SET_PATH = os.path.join(BASE_PATH, "QADataset", "ObliQA_MultiPassage_test.json")
QREL_PATH = os.path.join(BASE_PATH, "qrels.trec")

# --- Model & Embedding Input Folders ---
FINETUNED_RETRIEVER_FOLDER = os.path.join(BASE_PATH, "fine_tuned_retrievers")
ADVANCED_FINETUNED_FOLDER = os.path.join(BASE_PATH, "fine_tuned_retrievers_advanced")
EMBEDDINGS_FOLDER = os.path.join(BASE_PATH, "embeddings_full_comparison")

# --- Output Paths ---
RESULTS_CSV_OUTPUT_PATH = os.path.join(BASE_PATH, "experiment_2_final_graph_rerank_results.csv")
RESULTS_JSON_OUTPUT_FOLDER = os.path.join(BASE_PATH, "experiment_2_final_graph_rerank_json")
os.makedirs(RESULTS_JSON_OUTPUT_FOLDER, exist_ok=True)


# --- Champion Retriever Configurations (from final retriever evaluation) ---
CHAMPION_CONFIGS = [
    {
        "name": "e5-large-v2_FT_parent_child_cites",
        "model_key": "e5-large-v2_FT",
        "model_path": os.path.join(FINETUNED_RETRIEVER_FOLDER, "e5-large-v2"),
        "context_key": "parent_child_cites"
    },
    {
        "name": "e5-large-v2_FT_Advanced_parent",
        "model_key": "e5-large-v2_FT_Advanced",
        "model_path": os.path.join(ADVANCED_FINETUNED_FOLDER, "e5-large-v2"),
        "context_key": "parent"
    }
]


# --- Enriched Experiment Hyperparameters ---
K_INITIAL = 100
K_FINAL = 20
PARENT_BONUS_OPTIONS = [0.01, 0.02, 0.05]
CITES_BONUS_OPTIONS = [0.01, 0.02]
CITED_BY_BONUS_OPTIONS = [0.02, 0.04]
SIBLING_BONUS_OPTIONS = [0.005, 0.01]
PAGERANK_WEIGHT_OPTIONS = [0.0, 0.1, 0.2]
ISOLATION_PENALTY_OPTIONS = [0.0, -0.01]

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# --- Load Static Components ---
print("Loading static components...")
with open(GRAPH_PATH, "rb") as f: G = pickle.load(f)
with open(TEST_SET_PATH, "r", encoding="utf-8") as f: test_data = json.load(f)
qid_to_question = {q["QuestionID"]: q["Question"] for q in test_data}
print(f"Loaded {len(test_data)} test questions.")

# --- Pre-calculate PageRank ---
print("Pre-calculating PageRank for all graph nodes...")
pagerank_scores = nx.pagerank(G, alpha=0.85)
print("PageRank calculation complete.")

# Load QRELs
qrel = {}
with open(QREL_PATH, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split()
        qid, _, uid, rel = parts[0], parts[1], " ".join(parts[2:-1]), int(parts[-1])
        qrel.setdefault(qid, {})[uid] = rel
print(f"Loaded QRELs for {len(qrel)} queries.")

# Prepare BM25
print("Preparing BM25 index...")
all_passage_uids = [n for n, d in G.nodes(data=True) if d.get("type") == "Passage"]
uid_map = {f"{G.nodes[uid].get('document_id')}|||{G.nodes[uid].get('passage_id')}": uid for uid in all_passage_uids}
corpus_texts = [G.nodes[uid].get("text", "") for uid in all_passage_uids]
tokenized_corpus = [text.split() for text in corpus_texts]
bm25 = BM25Okapi(tokenized_corpus)
print("BM25 ready.")


# --- Helper Functions ---
def add_instruction_to_query(query, model_name):
    if "e5" in model_name or "bge" in model_name:
        return f"query: {query}"
    return query

def evaluate_run(run_dict, qrel_dict, metrics={"recall_10", "map_cut_10", "ndcg_cut_10"}):
    evaluator = pytrec_eval.RelevanceEvaluator(qrel_dict, metrics)
    results = evaluator.evaluate(run_dict)
    agg = {metric: pytrec_eval.compute_aggregated_measure(metric, [r.get(metric, 0.0) for r in results.values()]) for metric in metrics}
    return agg

def reciprocal_rank_fusion(ranked_lists, k=60):
    fused = {}
    for lst in ranked_lists:
        for rank, uid in enumerate(lst):
            fused[uid] = fused.get(uid, 0) + 1 / (k + rank + 1)
    return sorted(fused.items(), key=lambda x: x[1], reverse=True)

def format_run_for_json(run_dict, qid_to_question_map, uid_to_internal_uid_map, graph, top_n=10):
    output_list = []
    for qid, passages in run_dict.items():
        sorted_passages = sorted(passages.items(), key=lambda item: item[1], reverse=True)

        retrieved_passages_text, retrieved_scores, retrieved_ids = [], [], []
        for combined_uid, score in sorted_passages[:top_n]:
            internal_uid = uid_to_internal_uid_map.get(combined_uid)
            if internal_uid:
                retrieved_passages_text.append(graph.nodes[internal_uid].get("text", ""))
                retrieved_scores.append(score)
                retrieved_ids.append(internal_uid)

        output_list.append({
            "QuestionID": qid, "Question": qid_to_question_map.get(qid, ""),
            "RetrievedPassages": retrieved_passages_text, "RetrievedScores": retrieved_scores,
            "RetrievedIDs": retrieved_ids
        })
    return output_list

# --- Main Experiment Loop ---
all_results = []
hyperparameter_combinations = list(itertools.product(
    PARENT_BONUS_OPTIONS, CITES_BONUS_OPTIONS, CITED_BY_BONUS_OPTIONS,
    SIBLING_BONUS_OPTIONS, PAGERANK_WEIGHT_OPTIONS, ISOLATION_PENALTY_OPTIONS
))

for config in CHAMPION_CONFIGS:
    champion_name = config["name"]
    model_path = config["model_path"]
    model_key = config["model_key"]
    context_key = config["context_key"]

    print("\n" + "="*80)
    print(f"--- TESTING CHAMPION RETRIEVER: {champion_name} ---")
    print("="*80)

    # Load Embeddings & Query Encoder
    print(f"Loading embeddings from: {EMBEDDINGS_FOLDER}")
    emb_path = os.path.join(EMBEDDINGS_FOLDER, model_key, context_key, "embeddings.pkl")
    id_path = os.path.join(EMBEDDINGS_FOLDER, model_key, context_key, "passage_ids.json")
    try:
        with open(emb_path, "rb") as f: passage_embeddings = pickle.load(f)
        with open(id_path, "r") as f: passage_ids = json.load(f)
        embeddings_tensor = torch.tensor(passage_embeddings).to(device)
        query_encoder = SentenceTransformer(model_path, device=device)
        print("Champion retriever components loaded successfully.")
    except FileNotFoundError:
        print(f"FATAL ERROR: Embeddings not found at {emb_path}. Skipping this champion.")
        continue

    # Pre-calculate initial retrievals
    print("Pre-calculating initial hybrid retrievals...")
    initial_retrievals = {}
    for q in tqdm(test_data, desc=f"Initial Retrieval for {champion_name}"):
        qid, query = q["QuestionID"], q["Question"]
        instructed_query = add_instruction_to_query(query, champion_name)
        query_emb = query_encoder.encode(instructed_query, convert_to_tensor=True, device=device)
        cos_scores = util.pytorch_cos_sim(query_emb, embeddings_tensor)[0]
        top_dense = torch.topk(cos_scores, k=min(K_INITIAL, len(passage_ids)))
        dense_uids = [passage_ids[idx] for idx in top_dense.indices]
        uid_to_initial_score = {passage_ids[idx]: score.item() for idx, score in zip(top_dense.indices, top_dense.values)}

        bm25_scores = bm25.get_scores(query.split())
        top_bm25 = np.argsort(bm25_scores)[::-1][:K_INITIAL]
        bm25_uids = [all_passage_uids[i] for i in top_bm25]

        fused_results = reciprocal_rank_fusion([dense_uids, bm25_uids])
        initial_retrievals[qid] = (fused_results, uid_to_initial_score)

    # Loop through hyperparameter combinations
    for params in tqdm(hyperparameter_combinations, desc=f"Evaluating Hyperparameters for {champion_name}"):
        parent_b, cites_b, cited_by_b, sibling_b, pagerank_w, isolation_p = params

        graph_reranked_run = {}
        for q in test_data:
            qid = q["QuestionID"]
            fused_results, uid_to_initial_score = initial_retrievals[qid]

            initial_candidates = [{"internal_uid": uid, "initial_score": uid_to_initial_score.get(uid, 0), "graph_bonus": 0.0} for uid, rrf_score in fused_results]
            retrieved_uids_set = {cand["internal_uid"] for cand in initial_candidates[:K_INITIAL]}

            candidates_to_rerank = initial_candidates[:K_INITIAL]
            uid_to_parents = {cand["internal_uid"]: [p for p in G.predecessors(cand["internal_uid"]) if G.get_edge_data(p, cand["internal_uid"], {}).get("type") == "PARENT_OF"] for cand in candidates_to_rerank}

            for i, cand in enumerate(candidates_to_rerank):
                current_uid = cand["internal_uid"]
                bonus_applied = False

                if cand["internal_uid"] in uid_to_parents and uid_to_parents[cand["internal_uid"]]:
                    parent_uid = uid_to_parents[cand["internal_uid"]][0]
                    if parent_uid in retrieved_uids_set:
                        cand["graph_bonus"] += parent_b
                        bonus_applied = True
                    for j, other_cand in enumerate(candidates_to_rerank):
                        if i != j and other_cand["internal_uid"] in uid_to_parents and uid_to_parents[other_cand["internal_uid"]] and uid_to_parents[other_cand["internal_uid"]][0] == parent_uid:
                            cand["graph_bonus"] += sibling_b
                            bonus_applied = True
                            break

                for neighbor_uid in G.successors(current_uid):
                    if G.get_edge_data(current_uid, neighbor_uid, {}).get("type") == "CITES" and neighbor_uid in retrieved_uids_set:
                        cand["graph_bonus"] += cites_b
                        bonus_applied = True

                for neighbor_uid in G.predecessors(current_uid):
                    if G.get_edge_data(neighbor_uid, current_uid, {}).get("type") == "CITED_BY" and neighbor_uid in retrieved_uids_set:
                        cand["graph_bonus"] += cited_by_b
                        bonus_applied = True

                if not bonus_applied:
                    cand["graph_bonus"] += isolation_p

            for cand in candidates_to_rerank:
                pagerank_bonus = pagerank_scores.get(cand["internal_uid"], 0.0) * pagerank_w
                cand["graph_score"] = cand["initial_score"] + cand["graph_bonus"] + pagerank_bonus

            graph_reranked = sorted(candidates_to_rerank, key=lambda x: x["graph_score"], reverse=True)

            graph_reranked_run[qid] = {}
            for cand in graph_reranked[:K_FINAL]:
                node = G.nodes[cand["internal_uid"]]
                combined_uid = f"{node.get('document_id')}|||{node.get('passage_id')}"
                graph_reranked_run[qid][combined_uid] = float(cand["graph_score"])

        graph_metrics = evaluate_run(graph_reranked_run, qrel)
        all_results.append({
            "champion_retriever": champion_name, "parent_bonus": parent_b, "cites_bonus": cites_b,
            "cited_by_bonus": cited_by_b, "sibling_bonus": sibling_b, "pagerank_weight": pagerank_w,
            "isolation_penalty": isolation_p, **graph_metrics
        })

        # Save JSON output for this run
        param_str = f"p{parent_b}_c{cites_b}_cb{cited_by_b}_s{sibling_b}_pg{pagerank_w}_ip{isolation_p}"
        json_output_path = os.path.join(RESULTS_JSON_OUTPUT_FOLDER, f"{champion_name}_{param_str}.json")
        json_data = format_run_for_json(graph_reranked_run, qid_to_question, uid_map, G)
        with open(json_output_path, 'w') as f:
            json.dump(json_data, f, indent=4)


# --- Save and Display Final Results ---
results_df = pd.DataFrame(all_results)
results_df = results_df.rename(columns={"recall_10": "Recall@10", "map_cut_10": "MAP@10"})
results_df = results_df.sort_values(by=["Recall@10", "MAP@10"], ascending=False)

print("\n📊 Final Evaluation Results:")
print(results_df.to_string(index=False, float_format="%.4f"))

results_df.to_csv(RESULTS_CSV_OUTPUT_PATH, index=False)
print(f"\n✅ CSV summary saved to: {RESULTS_CSV_OUTPUT_PATH}")
print(f"✅ Detailed JSON results saved to: {RESULTS_JSON_OUTPUT_FOLDER}")

print("\n--- 🏆 Best Performing Configuration ---")
print(results_df.iloc[0])


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 131.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00

Initial Retrieval for e5-large-v2_FT_parent_child_cites:   0%|          | 0/447 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Initial Retrieval for e5-large-v2_FT_parent_child_cites: 100%|██████████| 447/447 [01:07<00:00,  6.67it/s]
Evaluating Hyperparameters for e5-large-v2_FT_parent_child_cites: 100%|██████████| 144/144 [03:33<00:00,  1.48s/it]



--- TESTING CHAMPION RETRIEVER: e5-large-v2_FT_Advanced_parent ---
Loading embeddings from: /content/drive/MyDrive/Colab Notebooks/RIRAG-MultiPassage-NLLP/embeddings_full_comparison
Champion retriever components loaded successfully.
Pre-calculating initial hybrid retrievals...


Initial Retrieval for e5-large-v2_FT_Advanced_parent: 100%|██████████| 447/447 [01:04<00:00,  6.95it/s]
Evaluating Hyperparameters for e5-large-v2_FT_Advanced_parent: 100%|██████████| 144/144 [03:33<00:00,  1.48s/it]


📊 Final Evaluation Results:
               champion_retriever  parent_bonus  cites_bonus  cited_by_bonus  sibling_bonus  pagerank_weight  isolation_penalty  Recall@10  MAP@10  ndcg_cut_10
e5-large-v2_FT_parent_child_cites        0.0200       0.0100          0.0200         0.0050           0.0000            -0.0100     0.4380  0.2998       0.3880
e5-large-v2_FT_parent_child_cites        0.0200       0.0100          0.0200         0.0050           0.1000            -0.0100     0.4380  0.2998       0.3880
e5-large-v2_FT_parent_child_cites        0.0200       0.0100          0.0200         0.0050           0.2000            -0.0100     0.4380  0.2998       0.3880
e5-large-v2_FT_parent_child_cites        0.0200       0.0100          0.0400         0.0050           0.0000            -0.0100     0.4380  0.2998       0.3880
e5-large-v2_FT_parent_child_cites        0.0200       0.0100          0.0400         0.0050           0.1000            -0.0100     0.4380  0.2998       0.3880
e5-large-v2

--- 🏆 Best Performing Configuration ---
champion_retriever    e5-large-v2_FT_parent_child_cites
parent_bonus                                       0.02
cites_bonus                                        0.01
cited_by_bonus                                     0.02
sibling_bonus                                     0.005
pagerank_weight                                     0.0
isolation_penalty                                 -0.01
Recall@10                                      0.437957
MAP@10                                         0.299846
ndcg_cut_10                                    0.387992
Name: 49, dtype: object

Analysis of the Results
Let's compare the best-performing graph re-ranking configuration against the baseline performance of the champion retriever itself.

Baseline (Champion Retriever from Exp 1):

Model: e5-large-v2_FT with parent_child_cites context (Hybrid)

Recall@10: 0.4497

MAP@10: 0.3319

Best Graph Re-ranked (from this Exp 2 re-run):

Model: e5-large-v2_FT with parent_child_cites context + Graph Bonuses

Recall@10: 0.4380

MAP@10: 0.2998

Conclusion:

The results are definitive and confirm our earlier hypothesis: the heuristic graph re-ranking stage had a negative impact on performance.

Even when starting with a much stronger, fine-tuned retriever, the simple, rule-based bonuses for parents, citations, and PageRank were not nuanced enough to improve the ranking. In fact, they slightly decreased both the recall and the precision (MAP).

This is a valuable "negative result" for your research. It proves that a sophisticated initial retriever is more effective on its own than when combined with a simple, additive re-ranking layer. This finding strongly justifies the need for a more powerful, content-aware re-ranker.

In [3]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
  #display(Audio(url='https://www.myinstants.com/media/sounds/anime-wow-sound-effect.mp3', autoplay=True))
  display(Audio(url='https://www.myinstants.com/media/sounds/money-soundfx.mp3', autoplay=True))
## Insert whatever audio file you want above

allDone()